## Pre-MCMC calculations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import astropy
import scipy.integrate as integrate
import pandas

In [2]:
# Importing constants 

Planck = astropy.cosmology.realizations.Planck18

T_CMB = Planck.Tcmb0  # Temperature of the CMB
H_0 = Planck.H0  # Current Hubble constant
Omega_m = Planck.Om0  # Matter density parameter
Omega_lambda = Planck.Ode0  # Dark energy density parameter
c = astropy.constants.c.to('km/s')  # Speed of light in km/s
sigma_T = astropy.constants.sigma_T.to('km2')  # Thomson scattering cross-section


# User set parameters

z = 8  # Redshift of the quasar bubbles 
f_H = 0.21  # fraction of neutral hydrogen at z = 8

In [3]:
# Input Data

data = pandas.read_csv('Data/l_0.0pc_noise.csv')

Ang_deg = np.array(data['radius_y'])  # In arcmins
amp = np.array(data['amp_y'])  # Amplitude in micro K

In [4]:
# Calculating the relevant distances

D_H = c / H_0  # Hubble distance

def integrand(z, Omega_m = Omega_m, Omega_lambda = Omega_lambda):
    return 1 / np.sqrt(Omega_m * (1 + z)**3 + Omega_lambda)

Integral = integrate.quad(integrand, 0, z)[0]  # Integrating from 0 to z
D_C = D_H * Integral  # Comoving distance
D_A = D_C / (1 + z)  # Angular diameter distance 

In [5]:
# Calculating the actual diameter of the bubbles and y

Ang_rad = Ang_deg * np.pi / (180 * 60)  # Convert arcmins to radians
Radius = D_A * Ang_rad  # Convert angular radius to actual radius
Diameter = 2 * Radius  # Diameter of the bubbles

y = amp / T_CMB.value  

In [6]:
# Calculating n_e and n_H

n_e = y * 3 * np.sqrt(1+z) / (sigma_T * 0.001 * 2 * Diameter.to('km'))

n_H = n_e / (1 - f_H)  # Assuming all electrons are from ionized hydrogen

----

## MCMC Sampling

In [7]:
# Defining the model using the Stromgren sphere approximation

def model(theta, n_H): 
    N_dot, t_Q = theta
    return (((3*N_dot*t_Q)/(4*np.pi*n_H))**(1/3)) * (1+z)**(-1)

In [8]:
# Setting up initial, lower and upper bounds for the parameters

N_dot_ini = 10**58  # in s^-1
N_dot_min =  10**56  # in s^-1
N_dot_max = 10**60  # in s^-1

t_Q_ini = 10**7  # in years
t_Q_ini = t_Q_ini * 3.156e7  # in s
t_Q_min = 10**5  # in years
t_Q_min = t_Q_min * 3.156e7  # in s 
t_Q_max = 10**9  # in years
t_Q_max = t_Q_max * 3.156e7  # in s

In [9]:
# Calculating error in the diameter

Ang_err = np.array([0.1] * len(Ang_deg))  # Error in determining the angular diameter in arcmins
Diam_err = Ang_err * np.pi / (180 * 60) * D_A * 2  # Convert angular error to actual diameter error

In [10]:
# Defining the log-likelihood function

def lnlike(theta, x, y, y_err):
    return -0.5 * np.sum(((y - model(theta, x)) / y_err) ** 2)

# Defining the log-prior function

def lnprior(theta):
    N_dot, t_Q = theta
    if N_dot < N_dot_min or t_Q < t_Q_min: # or N_dot > N_dot_max  or t_Q > t_Q_max:
        return -np.inf
    return 0.0

# Defining the log-posterior function

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)


In [11]:
# Setting initial values for the parameters
initial = [N_dot_ini, t_Q_ini]  

# Setting up the MCMC sampler
ndim = len(initial)
nwalkers = 5 * ndim    # Preferentially 3-5 times the number of dimensions (minimum 2 times the number of dimensions)
n_burn = 2000
n_steps = 100000

# Initializing the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(n_H, Diameter, Diam_err))

# Adding little variation to the initial positions
p0 = []
for i in range(nwalkers):
    pos = [N_dot_ini, t_Q_ini]  # Start with the initial values
    pos[0] = pos[0] + 1e-4 * pos[0] * np.random.random(1)[0]  # Adding a small random variation to N_dot
    pos[1] = pos[1] + 1e-4 * pos[1] * np.random.random(1)[0]  # Adding a small random variation to t_Q
    p0.append(pos)   


In [12]:
# Running the MCMC sampler

# Burn-in phase
print("Running burn-in phase...")
sampler.reset()
state = sampler.run_mcmc(p0, n_burn, progress=True)
sampler.reset()

# Main sampling phase
print("Running main sampling phase...")
pos, prob, state = sampler.run_mcmc(state, n_steps, progress=True)

# Extracting the samples
samples = sampler.get_chain(flat=True)      # Flatten the chain to get all samples in a single array


Running burn-in phase...


  2%|▏         | 30/2000 [00:00<00:06, 294.95it/s]

100%|██████████| 2000/2000 [00:06<00:00, 304.23it/s]


Running main sampling phase...


100%|██████████| 100000/100000 [05:33<00:00, 300.08it/s]


In [13]:
theta_max = samples[np.argmax(sampler.flatlnprobability)]
print(theta_max)

[3.42400371e+63 1.02900391e+20]


In [ ]:
'''
res = [2.05235821e+63 1.75222496e+20], [2.95231565e+63 1.21809305e+20]
[1.50480365e+64 2.38981167e+19]
[2.87363174e+64 1.25144664e+19]
[4.11184043e+64 8.74592398e+18]
[5.32163540e+63 6.75769181e+19]
[3.42400371e+63 1.02900391e+20]
'''

'\nres = [2.05235821e+63 1.75222496e+20], [2.95231565e+63 1.21809305e+20]\n[1.50480365e+64 2.38981167e+19]\n[2.87363174e+64 1.25144664e+19]\n[4.11184043e+64 8.74592398e+18]\n[5.32163540e+63 6.75769181e+19]\n'